## Query source LH, save to target LH + adjust to semantic model
This notebook queries the source lakehouse based on a defined SQL query. After that, it saves the result set to the target lakehouse. 

Next, it uses Semantic Link Labs to add the resulting table from the target lakehouse to the defined semantic model. 

In [1]:
%pip install semantic-link-labs
import sempy_labs as labs

StatementMeta(, b8b273bc-3cdf-40c0-9b83-d676edaf4aa5, 8, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.5/713.5 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 210.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: semantic-link-sempy
    Found existing installation: semantic-link-sempy 0.9.3
    Not uninstalling semantic-link-sempy at /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages, outside environment /nfs4/pyenv-7dc0401a-c14b-482e-aae6-bee13b4075d5
    Can't uninstall 'semantic-link-sempy'. No files were found to uninstall.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [2]:
# Define source and target lakehouses (schemas/databases)
source_lakehouse = "LH_STORE_Silver"
target_lakehouse = "LH_STORE_Gold"

StatementMeta(, b8b273bc-3cdf-40c0-9b83-d676edaf4aa5, 10, Finished, Available, Finished)

#### Create table in destination lakehouse

In [3]:
# Define source table and target table names
source_table = "date"
target_table = "date"  # new table in Gold to save results

StatementMeta(, b8b273bc-3cdf-40c0-9b83-d676edaf4aa5, 11, Finished, Available, Finished)

In [4]:
sql_query = f"""
SELECT 
    DateKey, 
    FullDateAlternateKey AS Date, 
    FiscalYear,
    FiscalQuarter,
    EnglishMonthName AS MonthName
FROM {source_lakehouse}.{source_table}
"""

# Run the SQL query and get a DataFrame
df = spark.sql(sql_query)

# Show first 5 rows of the resulting DataFrame
df.show(5)


StatementMeta(, b8b273bc-3cdf-40c0-9b83-d676edaf4aa5, 12, Finished, Available, Finished)

+--------+-------------------+----------+-------------+---------+
| DateKey|               Date|FiscalYear|FiscalQuarter|MonthName|
+--------+-------------------+----------+-------------+---------+
|20210101|2021-01-01 00:00:00|      2021|            3|  January|
|20210108|2021-01-08 00:00:00|      2021|            3|  January|
|20210115|2021-01-15 00:00:00|      2021|            3|  January|
|20210122|2021-01-22 00:00:00|      2021|            3|  January|
|20210129|2021-01-29 00:00:00|      2021|            3|  January|
+--------+-------------------+----------+-------------+---------+
only showing top 5 rows



In [5]:
# Write the DataFrame as a new table in the target lakehouse
df.write.mode("overwrite").saveAsTable(f"{target_lakehouse}.{target_table}")

print(f"✅ Saved filtered data to table '{target_table}' in lakehouse '{target_lakehouse}'")

StatementMeta(, b8b273bc-3cdf-40c0-9b83-d676edaf4aa5, 13, Finished, Available, Finished)

✅ Saved filtered data to table 'date' in lakehouse 'LH_STORE_Gold'


#### Update Semantic Model

In [6]:
semanticmodel_name = "Sales Analysis"

StatementMeta(, b8b273bc-3cdf-40c0-9b83-d676edaf4aa5, 14, Finished, Available, Finished)

In [7]:
labs.directlake.add_table_to_direct_lake_semantic_model(
    dataset= semanticmodel_name,
    table_name= target_table, # reusing the target table name, as the gold lakehouse should already be self explanatory
    lakehouse_table_name= target_table,
    refresh= False,
    workspace= None # if not specified, will be the same workspace as the notebook runs
)

StatementMeta(, b8b273bc-3cdf-40c0-9b83-d676edaf4aa5, 15, Finished, Available, Finished)

🟢 The 'date' table has been added to the 'Sales Analysis' semantic model within the 'Reaching maximum automation' workspace.
🟢 The 'date' partition has been added to the 'date' table in the 'Sales Analysis' semantic model within the 'Reaching maximum automation' workspace.
🟢 The 'DateKey' column has been added to the 'date' table as a 'Int64' data type in the 'Sales Analysis' semantic model within the 'Reaching maximum automation' workspace.
🟢 The 'Date' column has been added to the 'date' table as a 'DateTime' data type in the 'Sales Analysis' semantic model within the 'Reaching maximum automation' workspace.
🟢 The 'FiscalYear' column has been added to the 'date' table as a 'Int64' data type in the 'Sales Analysis' semantic model within the 'Reaching maximum automation' workspace.
🟢 The 'FiscalQuarter' column has been added to the 'date' table as a 'Int64' data type in the 'Sales Analysis' semantic model within the 'Reaching maximum automation' workspace.
🟢 The 'MonthName' column has 